In [12]:
import pandas as pd
from pandasql import sqldf
import numpy as np



1731. THe number of EMployees Which Report to Each Employee
Table: Employees

+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| reports_to  | int      |
| age         | int      |
+-------------+----------+
employee_id is the column with unique values for this table.
This table contains information about the employees and the id of the manager they report to. Some employees do not report to anyone (reports_to is null). 
 
For this problem, we will consider a manager an employee who has at least 1 other employee reporting to them.

Write a solution to report the ids and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by employee_id.

The result format is in the following example.

Input: 
Employees table:
+-------------+---------+------------+-----+ 
| employee_id | name    | reports_to | age |
|-------------|---------|------------|-----|
| 1           | Michael | null       | 45  |
| 2           | Alice   | 1          | 38  |
| 3           | Bob     | 1          | 42  |
| 4           | Charlie | 2          | 34  |
| 5           | David   | 2          | 40  |
| 6           | Eve     | 3          | 37  |
| 7           | Frank   | null       | 50  |
| 8           | Grace   | null       | 48  |
+-------------+---------+------------+-----+ 
Output: 
+-------------+---------+---------------+-------------+
| employee_id | name    | reports_count | average_age |
| ----------- | ------- | ------------- | ----------- |
| 1           | Michael | 2             | 40          |
| 2           | Alice   | 2             | 37          |
| 3           | Bob     | 1             | 37          |
+-------------+---------+---------------+-------------+

In [19]:
df = pd.read_csv('data/1731_employees.csv') # loads the example

,employee_id,name,reports_to,age
0,1,Michael,NaN,45
1,2,Alice,1.0,38
2,3,Bob,1.0,42
3,4,Charlie,2.0,34
4,5,David,2.0,40
5,6,Eve,3.0,37
6,7,Frank,NaN,50
7,8,Grace,NaN,48


In [60]:
#aux = df.dropna(subset=['reports_to'])

aux = df.copy()
#aux.loc[:,'average_age'] = aux.groupby('reports_to')['age'].transform('mean')

# Group employees by their manager to calculate the count of reports and the 
# average age
managers = aux.groupby('reports_to').agg(
    average_age = ('age', 'mean'),
    reports_count = ('name','count')
    ).reset_index()
managers.rename( columns = {'reports_to': 'employee_id' }, inplace = True)

# Adjust for banker's rounding by adding a very small number before rounding
managers[['average_age']] = (managers[['average_age']] + 1e-12).round()

# Merge the aggregated data with the original employees DataFrame to get the 
# names of managers
managers.merge(
    df,
    on = 'employee_id',
    how = 'left'
)[['employee_id', 'name', 'reports_count', 'average_age']]




,employee_id,name,report_count,average_age
0,1.0,Michael,2,40.0
1,2.0,Alice,2,37.0
2,3.0,Bob,1,37.0


In [46]:
# Create Users table
Users = pd.DataFrame( {
    'user_id': [1, 2, 3, 4],
    'join_date': ['2018-01-01', '2018-02-09', '2018-01-19', '2018-05-21'],
    'favorite_brand': ['Lenovo', 'Samsung', 'LG', 'HP']
})



# Create Orders table
Orders = pd.DataFrame( {
    'order_id': [1, 2, 3, 4, 5, 6],
    'order_date': ['2019-08-01', '2018-08-02', '2019-08-03', '2018-08-04', '2018-08-04', '2019-08-05'],
    'item_id': [4, 2, 3, 1, 1, 2],
    'buyer_id': [1, 1, 2, 4, 3, 2],
    'seller_id': [2, 3, 3, 2, 4, 4]
})



In [48]:
Orders

,order_id,order_date,item_id,buyer_id,seller_id
0,1,2019-08-01,4,1,2
1,2,2018-08-02,2,1,3
2,3,2019-08-03,3,2,3
3,4,2018-08-04,1,4,2
4,5,2018-08-04,1,3,4
5,6,2019-08-05,2,2,4


In [49]:
Users

,user_id,join_date,favorite_brand
0,1,2018-01-01,Lenovo
1,2,2018-02-09,Samsung
2,3,2018-01-19,LG
3,4,2018-05-21,HP


In [51]:
query = " SELECT * FROM Users"
pd.read_sql(query, con)

TypeError: read_sql() missing 1 required positional argument: 'con'

In [54]:
sqldf(''' 
    

SELECT 
  u.user_id AS buyer_id, 
  u.join_date, 
  COUNT(o.order_id) AS orders_in_2019 
FROM 
  Users u 
  LEFT JOIN Orders o 
    ON u.user_id = o.buyer_id AND o.order_date LIKE '2019%' 
GROUP BY 
  u.user_id 

 ''')

# Notice that 
# •	In this query, the filtering condition (o.order_date LIKE '2019%') is part of the ON clause 
# of the LEFT JOIN.
#	•	This means that the query will try to match Users with Orders where order_date starts with 
# 2019. If there’s a match, it will return those rows, but if there is no order for a user or
#  no order from 2019, the user will still be included in the result with a NULL value for
#  o.order_id (due to the LEFT JOIN).

# If i had, instead, used a WHERE conditions, I'd filter after the join, removing the observations 
# where orders_in_2019 = 0

,buyer_id,join_date,orders_in_2019
0,1,2018-01-01,1
1,2,2018-02-09,2
2,3,2018-01-19,0
3,4,2018-05-21,0


176. Second highest Salary

Table: Employee

+-------------+------+
| Column Name | Type |
+-------------+------+
| id          | int  |
| salary      | int  |
+-------------+------+
id is the primary key (column with unique values) for this table.
Each row of this table contains information about the salary of an employee.
 

Write a solution to find the second highest distinct salary from the Employee table. If there is no second highest salary, return null (return None in Pandas).

The result format is in the following example.

Example 1:

Input: 
Employee table:
+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
Output: 
+---------------------+
| SecondHighestSalary |
+---------------------+
| 200                 |
+---------------------+
Example 2:

Input: 
Employee table:
+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
+----+--------+
Output: 
+---------------------+
| SecondHighestSalary |
+---------------------+
| null                |
+---------------------+

In [72]:
employee = pd.DataFrame({
    'id': [1,2],
    'salary': [100, 100]
})

employee

,id,salary
0,1,100
1,2,100


In [73]:
sqldf(''' 
    

SELECT 
    max(salary) as SecondHighestSalary
FROM
    employee
where salary <(
    select max(salary) from Employee
)

 ''')


,SecondHighestSalary
0,None


In [64]:
# First, get only distinct salaries

aux = employee.drop_duplicates(['salary'])
aux = aux.sort_values('salary', ascending = False).reset_index()

if aux.shape[0] >= 2:
    aux['SecondHighestSalary'] = aux.loc[1,'salary']
    aux = aux[['SecondHighestSalary']].drop_duplicates()
else:
    aux['SecondHighestSalary'] = None

aux[['SecondHighestSalary']]



,SecondHighestSalary
0,200


### 1193. Monthly Transactions I


+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| country       | varchar |
| state         | enum    |
| amount        | int     |
| trans_date    | date    |
+---------------+---------+
id is the primary key of this table.
The table has information about incoming transactions.
The state column is an enum of type ["approved", "declined"].

Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.

Return the result table in any order.

The query result format is in the following example.

Input: 
Transactions table:
+------+---------+----------+--------+------------+
| id   | country | state    | amount | trans_date |
+------+---------+----------+--------+------------+
| 121  | US      | approved | 1000   | 2018-12-18 |
| 122  | US      | declined | 2000   | 2018-12-19 |
| 123  | US      | approved | 2000   | 2019-01-01 |
| 124  | DE      | approved | 2000   | 2019-01-07 |
+------+---------+----------+--------+------------+
Output: 
+----------+---------+-------------+----------------+--------------------+-----------------------+
| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
+----------+---------+-------------+----------------+--------------------+-----------------------+
| 2018-12  | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01  | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |
+----------+---------+-------------+----------------+--------------------+-----------------------+


In [35]:
df = pd.read_csv('data/transactions.csv')

# 1) Create column month
# 2) group by country/month 
# 3) COUNT transactions, SUM amount
# 4) COUNT transactions, SUM amount   IF state = approved

# 1) Create column month
   
df['month'] = df['trans_date'].dt.strftime("%Y-%m")
df['country'].fillna('null',inplace=True)  # ridiculous exception case

df['approved'] = np.where(df['state'] == 'approved', df['amount'], np.nan)

df.groupby(['month', 'country']).agg(
        trans_count=('state', 'count'),
        approved_count=('approved', 'count'),
        trans_total_amount=('amount', 'sum'),
        approved_total_amount=('approved', 'sum')
    ).reset_index()
        




,country,month,trans_count,trans_total_amount,approved_count,approved_total_amount
0,US,2018-12,2,3000,1,1000
1,DE,2019-01,1,2000,1,2000
2,US,2019-01,1,2000,1,2000


In [40]:
df = pd.read_csv('data/transactions.csv')
sqldf(''' 
	SELECT  
		SUBSTR(trans_date,1,7) as month, 
		country, 
		count(id) as trans_count, 
		SUM(state ='approved') as approved_count, 
		SUM(amount) as trans_total_amount, 
		SUM((state = 'approved') * amount) as approved_total_amount
	FROM df
	GROUP BY month, country
	''')

,month,country,trans_count,approved_count,trans_total_amount,approved_total_amount
0,2018-12,US,2,1,3000,1000
1,2019-01,DE,1,1,2000,2000
2,2019-01,US,1,1,2000,2000
